In [1]:
%run base_test.ipynb

from base import ml

In [2]:
gen_one_data = ml.gen_peakons_on_grid

In [3]:
dir_ = '../saved_models/grid_multi_peakon'
dnn_factories, names = ml.dnn_factories_from_dir(dir_)
extra_facs = []
extra_names = []

# average_reg = ml.RegressorAverager(regressor_factories=dnn_factories)
# average_reg.auto_mask(gen_one_data=ml.gen_peakons_on_grid, thresh=0.5, batch_size=1000)
# average_reg_fac = ml.RegressorFactory(regressor=average_reg)
# extra_facs.append(average_reg_fac)
# extra_names.append('AverageReg')

# Warning: the polynomial interpolation takes a while.
# poly_deg = 10
# poly_fac = ml.RegressorFactory(regressor=ml.FineGridPolyInterp(poly_deg=poly_deg))
# bilin_fac = ml.RegressorFactory(regressor=ml.FineGridBilinearInterp())
# extra_facs.extend([poly_fac, bilin_fac])
# extra_names.extend(['Poly{}Reg'.format(poly_deg), 'BilinearReg'])

results = ml.eval_regressors([*dnn_factories, 
                              *extra_facs], 
                             gen_one_data, 
                             1000)

for stat in ('loss', 'max_deviation'):
    print(stat)
    print('-' * len(stat))
    for result, name in sorted(zip(results, [*names,
                                             *extra_names]), 
                               key=lambda x: x[0][stat]):
        print(name, result[stat])
    print('')

INFO:tensorflow:Using config: {'_model_dir': '../saved_models/grid_multi_peakon/100x10_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbd9cfd3668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../saved_models/grid_multi_peakon/100x10_121__ScaleCenter_crelu/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init

In [4]:
for stat in ('loss', 'max_deviation'):
    print(stat)
    print('-' * len(stat))
    for result, name in sorted(zip(results, [*names,
                                             *extra_names]), 
                               key=lambda x: x[0][stat]):
        print(eval(name.split('_')[0].replace('x', '*')), result[stat])
    print('')

loss
----
2000 0.02187747618128558
660 0.023170463197898467
3000 0.023502808275558618
880 0.024358840579182493
720 0.02449292147683889
2400 0.02522905297147429
1000 0.026886969427828245
2400 0.02735637782222654
540 0.03240846104834928
600 0.03318183277977333
770 0.033403293566532224
1200 0.03386794986490695
700 0.034130948993964995
800 0.03529782208667189
960 0.03558736330396401
640 0.037811877337396084
560 0.03793416290995898
1600 0.0382603786047163
800 0.03883847966736061
3200 0.0394632713806486
840 0.040798930300974104
1600 0.04146516128364893
720 0.04182650792504797
800 0.04196179330226391
1200 0.04198069877260579
630 0.0421692226689673
1800 0.044024187586067146
4000 0.05100089291427229
480 0.051648123959676995
400 0.0529114886270829
400 0.054308343683738346
500 0.05575740092174509
400 0.06281394873287917
600 0.0749869359176176
100 0.07696053610230408
300 0.07821909375711594
200 0.09300559655021656
200 0.14145875193992877

max_deviation
-------------
3000 0.26960324690682036
660 0.

In [5]:
for stat in ('loss', 'max_deviation'):
    print(stat)
    print('-' * len(stat))
    best = sorted(zip(results, [*names, *extra_names]), 
                   key=lambda x: x[0][stat])[:10]
    for result, name in best:
        print(name)

loss
----
200x10_121__ScaleCenter_crelu
110x6_121__ScaleCenter_crelu
300x10_121__ScaleCenter_crelu
110x8_121__ScaleCenter_crelu
120x6_121__ScaleCenter_crelu
400x6_121__ScaleCenter_crelu
100x10_121__ScaleCenter_crelu
300x8_121__ScaleCenter_crelu
90x6_121__ScaleCenter_crelu
300x2_121__ScaleCenter_crelu
max_deviation
-------------
300x10_121__ScaleCenter_crelu
110x6_121__ScaleCenter_crelu
100x10_121__ScaleCenter_crelu
110x8_121__ScaleCenter_crelu
120x6_121__ScaleCenter_crelu
200x10_121__ScaleCenter_crelu
200x8_121__ScaleCenter_crelu
80x7_121__ScaleCenter_crelu
400x8_121__ScaleCenter_crelu
90x6_121__ScaleCenter_crelu
